**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
######### 
pip install scikit-video


     |████████████████████████████████| 2.3MB 3.4MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import pandas as pd

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras import losses
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten
from tqdm import tqdm_notebook as tqdmnb
import time

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Here the action is choosen according to the $\epsilon$-greedy policy:
* with probability $\epsilon$ the random action is taken, which means we follow *exploration* strategy.
* with probability $1-\epsilon$ the learned action is taken (that corresponding $a^*$ maximizing $Q$-function), which means we follow *exploitation* strategy using knowledge from previous experiences.

Thus the parameter $\epsilon$ defines exploration/exploitation contributions to the learning strategy. If $\epsilon=1$, the agent always does some random actions, it doesn't use its accumulated knowlegde to make a better choise. If $\epsilon=0$, the agent actions are too overfited to its previous experience, it stays at a local maximum of $Q$-function. So it is important to maintain the exploration/exploitation balance.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

* ```position``` contains a grid of size (grid_size+2)x(grid_size+2). Here padding contains -1 in its cells and is needed to provide the correct visual fiels when the rat is near the boards. The cell corresponding rat's current position contains value $1$, and the other cells - $0$. So the ```position``` array is for keeping the current rat's position.

* ```board``` is also a grid of the same size, but it contains rewards (positive or negative) in some cells, and $0$ in other cells. When the reward in a cell was taken, the number in ```board``` array corresponding to this cell changes to $0$. So the ```board``` contains all available rewards.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action+1, 1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
    
        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return score/epochs

In [30]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/9.0. Average score (3.0)
Win/lose count 8.0/8.0. Average score (1.5)
Win/lose count 6.5/11.0. Average score (-0.5)
Win/lose count 9.5/10.0. Average score (-0.5)
Win/lose count 9.5/9.0. Average score (-0.3)
Win/lose count 10.5/16.0. Average score (-1.1666666666666667)
Win/lose count 10.0/16.0. Average score (-1.8571428571428572)
Win/lose count 8.5/10.0. Average score (-1.8125)
Win/lose count 9.0/8.0. Average score (-1.5)
Win/lose count 5.5/4.0. Average score (-1.2)
Final score: -1.2


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
* We need to show that: 
\begin{equation*}
Q^{\pi}(s,a)=\mathbb E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
To prove that, we can start by the definition of $Q$-function:
\begin{align*}
Q^\pi(s,a)
&=
\mathbb E_{p^{\pi}}[\sum_{t=0}^{\infty}{\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a ]}
&& \text{definition}
\\&=
\mathbb E_{p^{\pi}}[r(s_{0},a_{0}) + \sum_{t=1}^{\infty}{\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a ]}
&& \text{rearranging terms in sum}
\\&=
r(s,a) + \gamma \mathbb E_{p^{\pi}}[\sum_{t=1}^{\infty}{\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a ]} 
\\&=
r(s,a) + \gamma \sum_{s^\prime,a^\prime}\mathbb P(s_1=s^\prime,a_1=a^\prime|s_{0}=s,a_{0}=a)\mathbb E_{p^{\pi}}[\sum_{t=1}^{\infty}{\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s^\prime,a_{1}=a^\prime ]} 
&& \text{Markov propertiy}
\\&= 
r(s,a) + \gamma \sum_{s^\prime,a^\prime}p(s^\prime,a^\prime|s,a)\mathbb E_{p^{\pi}}[\sum_{t^\prime=0}^{\infty}{\gamma^{t^\prime}r(s_{t},a_{t})|s_{0^\prime}=s^\prime,a_{0^\prime}=a^\prime ]} 
&& \text{MDP and change of time variable}
\\&= 
r(s,a) + \gamma \sum_{s^\prime,a^\prime}p(s^\prime,a^\prime|s,a)Q^\pi(s^\prime,a^\prime)
\\&= 
\mathbb E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

* Then we need to show that:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Starting with the definition of optimal policy:
\begin{align*}
Q^*(s,a)
&=
\max_{\pi}(Q^\pi(s,a))
&& \text{definition}
\\&=
\max_{a^\prime}(Q^{\pi^*}(s,a^\prime))
&& \text{where $Q^{\pi^*}(s,a) = Q^{\pi^*}(s,\pi^*(s))$}
\\&=
\max_{a^\prime}(\mathbb E_{s'\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')])
&& \text{from previous proof}
\\&=
\mathbb E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a^\prime}(Q^{*}(s',a')])
&& \text{move max to the term dependent on $a^\prime$}
\end{align*}

* Finaly, we need to deduce that the following expression is a plausible objective:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
This function converges to 0 if the policy we learned converges to the optimal one. In more details: consider Q-function parametrized by $\theta$, we minimize square loss $(Q^*(s,a)-Q_\theta(s,a))^2$, where $Q^*(s,a)$ is defined by the Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        rand_idx = np.random.randint(len(self.memory))
        return self.memory[rand_idx]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None,:]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            target_q[i] = self.model.predict(s_[None,:])
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
                input_states[i] = s_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(n_s_[None,:]), axis=1)
                input_states[i] = s_
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, hidden_size=30, num_layers=1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(hidden_size, activation="tanh"))
        for i in range(num_layers-1):
          model.add(Dense(hidden_size, activation="tanh"))
        model.add(Dense(self.n_action))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [42]:
epochs_train = 10
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/010 | Loss 0.0187 | Win/lose count 5.5/3.0 (2.5)
Epoch 001/010 | Loss 0.0038 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/010 | Loss 0.0081 | Win/lose count 4.5/8.0 (-3.5)
Epoch 003/010 | Loss 0.0083 | Win/lose count 9.5/6.0 (3.5)
Epoch 004/010 | Loss 0.0070 | Win/lose count 5.5/5.0 (0.5)
Epoch 005/010 | Loss 0.0087 | Win/lose count 7.5/3.0 (4.5)
Epoch 006/010 | Loss 0.0168 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/010 | Loss 0.0110 | Win/lose count 8.0/3.0 (5.0)
Epoch 008/010 | Loss 0.0053 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/010 | Loss 0.0044 | Win/lose count 4.0/3.0 (1.0)


***
We want to find optimal hyper-parameters. As training takes much time, I'll find an optimal architecture in sence of number of layers and number of hidden neurons in them. Other parameters are fixed.
***

In [44]:
epochs_train = 50
epochs_test = 30
fullconnect_res = []
layers = [1, 3]
hid_neurons = [30, 50]

for layer in tqdmnb(layers):
  for hidden in hid_neurons:
    env = Environment(grid_size=size, max_time=T, temperature=0.3)
    agent = DQN_FC(size, lr=lr, epsilon = 0.1, memory_size=ms, batch_size = bs, hidden_size=hidden, num_layers=layer)
    train(agent, env, epochs_train, prefix='fc_train')
    score = test(agent, env, epochs_test, prefix='fc_test')
    res = {'num_layers': layer, 'hidden_size': hidden,
            'learning_rate': 0.1, "batch_size": 32,
            "memory_size": 2000, 'score': score
            }
    print('result: {}'.format(res))
    fullconnect_res.append(res)

Epoch 000/050 | Loss 0.0005 | Win/lose count 1.5/0 (1.5)
Epoch 001/050 | Loss 0.0198 | Win/lose count 4.5/4.0 (0.5)
Epoch 002/050 | Loss 0.0296 | Win/lose count 6.5/13.0 (-6.5)
Epoch 003/050 | Loss 0.0395 | Win/lose count 4.0/11.0 (-7.0)
Epoch 004/050 | Loss 0.0035 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/050 | Loss 0.0345 | Win/lose count 1.5/3.0 (-1.5)
Epoch 006/050 | Loss 0.0085 | Win/lose count 3.0/7.0 (-4.0)
Epoch 007/050 | Loss 0.0356 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/050 | Loss 0.0190 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/050 | Loss 0.1029 | Win/lose count 3.0/3.0 (0.0)
Epoch 010/050 | Loss 0.0054 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/050 | Loss 0.0037 | Win/lose count 2.5/0 (2.5)
Epoch 012/050 | Loss 0.0046 | Win/lose count 4.0/6.0 (-2.0)
Epoch 013/050 | Loss 0.0058 | Win/lose count 4.5/7.0 (-2.5)
Epoch 014/050 | Loss 0.0021 | Win/lose count 1.5/4.0 (-2.5)
Epoch 015/050 | Loss 0.0039 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/050 | Loss 0.0007 | Win/lose count 0.5/3

In [45]:
fc_df = pd.DataFrame(fullconnect_res)
fc_df

,num_layers,hidden_size,learning_rate,batch_size,memory_size,score
0,1,30,0.1,32,2000,1.616667
1,1,50,0.1,32,2000,9.050000
2,3,30,0.1,32,2000,5.800000
3,3,50,0.1,32,2000,6.666667


***
We conclude that an optimal architecture contains 1 inner fully connected layer with 50 hidden neurons. Let's find optimal learning rate, batch size and memory size:
***

In [46]:
epochs_train = 20
epochs_test = 10
fullconnect_res_params = []
learning_rates = [0.05, 0.1, 0.25]
batch_sizes = [16, 32, 48]
memory_sizes = [1000, 2000, 3000]

for lr in tqdmnb(learning_rates):
  for bs in tqdmnb(batch_sizes):
    for ms in tqdmnb(memory_sizes):
      env = Environment(grid_size=size, max_time=T, temperature=0.3)
      agent = DQN_FC(size, lr=lr, epsilon = 0.1, memory_size=ms, batch_size = bs, hidden_size=50, num_layers=1)
      train(agent, env, epochs_train, prefix='fc_train')
      score = test(agent, env, epochs_test, prefix='fc_test')
      res = {'num_layers': 1, 'hidden_size': 50,
              'learning_rate': lr, "batch_size": bs,
              "memory_size": ms, 'score': score
              }
      print('result: {}'.format(res))
      fullconnect_res_params.append(res)

Epoch 000/020 | Loss 0.0032 | Win/lose count 3.0/8.0 (-5.0)
Epoch 001/020 | Loss 0.0151 | Win/lose count 8.5/6.0 (2.5)
Epoch 002/020 | Loss 0.0098 | Win/lose count 3.5/1.0 (2.5)
Epoch 003/020 | Loss 0.0179 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/020 | Loss 0.0111 | Win/lose count 3.0/3.0 (0.0)
Epoch 005/020 | Loss 0.0887 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/020 | Loss 0.0074 | Win/lose count 5.5/5.0 (0.5)
Epoch 007/020 | Loss 0.0069 | Win/lose count 3.0/5.0 (-2.0)
Epoch 008/020 | Loss 0.0024 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/020 | Loss 0.0296 | Win/lose count 3.0/1.0 (2.0)
Epoch 010/020 | Loss 0.0087 | Win/lose count 4.0/2.0 (2.0)
Epoch 011/020 | Loss 0.0078 | Win/lose count 4.5/2.0 (2.5)
Epoch 012/020 | Loss 0.0057 | Win/lose count 7.5/2.0 (5.5)
Epoch 013/020 | Loss 0.0059 | Win/lose count 10.5/3.0 (7.5)
Epoch 014/020 | Loss 0.0157 | Win/lose count 8.0/5.0 (3.0)
Epoch 015/020 | Loss 0.0054 | Win/lose count 7.0/2.0 (5.0)
Epoch 016/020 | Loss 0.0146 | Win/lose count 2.0/0 (2

Epoch 000/020 | Loss 0.0129 | Win/lose count 2.0/5.0 (-3.0)
Epoch 001/020 | Loss 0.0033 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/020 | Loss 0.0222 | Win/lose count 0/4.0 (-4.0)
Epoch 003/020 | Loss 0.0133 | Win/lose count 8.5/2.0 (6.5)
Epoch 004/020 | Loss 0.0153 | Win/lose count 3.0/0 (3.0)
Epoch 005/020 | Loss 0.0099 | Win/lose count 1.0/2.0 (-1.0)
Epoch 006/020 | Loss 0.0059 | Win/lose count 3.0/2.0 (1.0)
Epoch 007/020 | Loss 0.0032 | Win/lose count 4.0/0 (4.0)
Epoch 008/020 | Loss 0.0137 | Win/lose count 3.0/3.0 (0.0)
Epoch 009/020 | Loss 0.0144 | Win/lose count 5.0/8.0 (-3.0)
Epoch 010/020 | Loss 0.0097 | Win/lose count 3.5/1.0 (2.5)
Epoch 011/020 | Loss 0.0153 | Win/lose count 6.0/8.0 (-2.0)
Epoch 012/020 | Loss 0.0217 | Win/lose count 1.5/3.0 (-1.5)
Epoch 013/020 | Loss 0.0166 | Win/lose count 1.5/7.0 (-5.5)
Epoch 014/020 | Loss 0.0101 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/020 | Loss 0.0533 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/020 | Loss 0.0489 | Win/lose count 7.0/4.0 (

Epoch 000/020 | Loss 0.0111 | Win/lose count 1.0/1.0 (0.0)
Epoch 001/020 | Loss 0.0059 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/020 | Loss 0.0058 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/020 | Loss 0.0064 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/020 | Loss 0.0012 | Win/lose count 7.0/4.0 (3.0)
Epoch 005/020 | Loss 0.0101 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/020 | Loss 0.0100 | Win/lose count 3.0/3.0 (0.0)
Epoch 007/020 | Loss 0.0224 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/020 | Loss 0.0111 | Win/lose count 2.0/2.0 (0.0)
Epoch 009/020 | Loss 0.0382 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/020 | Loss 0.0132 | Win/lose count 2.5/2.0 (0.5)
Epoch 011/020 | Loss 0.0120 | Win/lose count 1.5/1.0 (0.5)
Epoch 012/020 | Loss 0.0037 | Win/lose count 8.5/5.0 (3.5)
Epoch 013/020 | Loss 0.0156 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/020 | Loss 0.0375 | Win/lose count 13.5/3.0 (10.5)
Epoch 015/020 | Loss 0.0071 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/020 | Loss 0.0079 | Win/lose count 10.0/4.0 

Epoch 000/020 | Loss 0.0041 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/020 | Loss 0.0196 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/020 | Loss 0.0154 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/020 | Loss 0.0124 | Win/lose count 4.0/1.0 (3.0)
Epoch 004/020 | Loss 0.0014 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/020 | Loss 0.0116 | Win/lose count 2.5/3.0 (-0.5)
Epoch 006/020 | Loss 0.0010 | Win/lose count 2.5/5.0 (-2.5)
Epoch 007/020 | Loss 0.0025 | Win/lose count 5.0/0 (5.0)
Epoch 008/020 | Loss 0.0041 | Win/lose count 9.5/10.0 (-0.5)
Epoch 009/020 | Loss 0.0026 | Win/lose count 3.0/0 (3.0)
Epoch 010/020 | Loss 0.0116 | Win/lose count 2.5/4.0 (-1.5)
Epoch 011/020 | Loss 0.0013 | Win/lose count 7.0/3.0 (4.0)
Epoch 012/020 | Loss 0.1027 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/020 | Loss 0.0024 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/020 | Loss 0.0021 | Win/lose count 3.5/1.0 (2.5)
Epoch 015/020 | Loss 0.0021 | Win/lose count 6.0/6.0 (0.0)
Epoch 016/020 | Loss 0.0956 | Win/lose count 12.5/3.0 

Epoch 000/020 | Loss 0.0117 | Win/lose count 2.5/1.0 (1.5)
Epoch 001/020 | Loss 0.0073 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/020 | Loss 0.0117 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/020 | Loss 0.0031 | Win/lose count 1.5/5.0 (-3.5)
Epoch 004/020 | Loss 0.0030 | Win/lose count 5.5/2.0 (3.5)
Epoch 005/020 | Loss 0.0640 | Win/lose count 5.0/6.0 (-1.0)
Epoch 006/020 | Loss 0.0130 | Win/lose count 0.5/3.0 (-2.5)
Epoch 007/020 | Loss 0.0098 | Win/lose count 5.0/6.0 (-1.0)
Epoch 008/020 | Loss 0.0042 | Win/lose count 4.5/5.0 (-0.5)
Epoch 009/020 | Loss 0.0043 | Win/lose count 3.5/4.0 (-0.5)
Epoch 010/020 | Loss 0.0037 | Win/lose count 2.0/0 (2.0)
Epoch 011/020 | Loss 0.0599 | Win/lose count 4.0/3.0 (1.0)
Epoch 012/020 | Loss 0.0081 | Win/lose count 5.5/2.0 (3.5)
Epoch 013/020 | Loss 0.0040 | Win/lose count 4.0/0 (4.0)
Epoch 014/020 | Loss 0.0059 | Win/lose count 3.5/1.0 (2.5)
Epoch 015/020 | Loss 0.0080 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/020 | Loss 0.0019 | Win/lose count 5.0/3.0 

Epoch 000/020 | Loss 0.0178 | Win/lose count 6.5/8.0 (-1.5)
Epoch 001/020 | Loss 0.0117 | Win/lose count 4.5/7.0 (-2.5)
Epoch 002/020 | Loss 0.0118 | Win/lose count 6.0/6.0 (0.0)
Epoch 003/020 | Loss 0.0563 | Win/lose count 4.0/8.0 (-4.0)
Epoch 004/020 | Loss 0.0078 | Win/lose count 6.0/3.0 (3.0)
Epoch 005/020 | Loss 0.0048 | Win/lose count 2.5/6.0 (-3.5)
Epoch 006/020 | Loss 0.0575 | Win/lose count 4.0/7.0 (-3.0)
Epoch 007/020 | Loss 0.0094 | Win/lose count 5.0/4.0 (1.0)
Epoch 008/020 | Loss 0.0036 | Win/lose count 0.5/0 (0.5)
Epoch 009/020 | Loss 0.0037 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/020 | Loss 0.0067 | Win/lose count 7.5/0 (7.5)
Epoch 011/020 | Loss 0.0038 | Win/lose count 5.0/0 (5.0)
Epoch 012/020 | Loss 0.0425 | Win/lose count 2.5/0 (2.5)
Epoch 013/020 | Loss 0.0030 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/020 | Loss 0.0052 | Win/lose count 10.0/2.0 (8.0)
Epoch 015/020 | Loss 0.0424 | Win/lose count 10.5/1.0 (9.5)
Epoch 016/020 | Loss 0.0051 | Win/lose count 8.0/4.0 (4.

Epoch 000/020 | Loss 0.0036 | Win/lose count 1.0/3.0 (-2.0)
Epoch 001/020 | Loss 0.0072 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/020 | Loss 0.0063 | Win/lose count 1.0/4.0 (-3.0)
Epoch 003/020 | Loss 0.1597 | Win/lose count 5.0/3.0 (2.0)
Epoch 004/020 | Loss 0.0033 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/020 | Loss 0.0068 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/020 | Loss 0.0033 | Win/lose count 7.0/3.0 (4.0)
Epoch 007/020 | Loss 0.0054 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/020 | Loss 0.0021 | Win/lose count 3.5/2.0 (1.5)
Epoch 009/020 | Loss 0.0050 | Win/lose count 5.5/1.0 (4.5)
Epoch 010/020 | Loss 0.0091 | Win/lose count 3.5/1.0 (2.5)
Epoch 011/020 | Loss 0.0019 | Win/lose count 2.0/1.0 (1.0)
Epoch 012/020 | Loss 0.0046 | Win/lose count 4.0/4.0 (0.0)
Epoch 013/020 | Loss 0.0046 | Win/lose count 12.0/4.0 (8.0)
Epoch 014/020 | Loss 0.0033 | Win/lose count 3.0/4.0 (-1.0)
Epoch 015/020 | Loss 0.0004 | Win/lose count 6.5/3.0 (3.5)
Epoch 016/020 | Loss 0.0016 | Win/lose count 7.5/0

Epoch 000/020 | Loss 0.0072 | Win/lose count 2.0/8.0 (-6.0)
Epoch 001/020 | Loss 0.0044 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/020 | Loss 0.0058 | Win/lose count 12.0/6.0 (6.0)
Epoch 003/020 | Loss 0.0309 | Win/lose count 8.0/4.0 (4.0)
Epoch 004/020 | Loss 0.0428 | Win/lose count 6.5/2.0 (4.5)
Epoch 005/020 | Loss 0.0056 | Win/lose count 8.5/7.0 (1.5)
Epoch 006/020 | Loss 0.0041 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/020 | Loss 0.0036 | Win/lose count 11.0/4.0 (7.0)
Epoch 008/020 | Loss 0.0026 | Win/lose count 15.5/2.0 (13.5)
Epoch 009/020 | Loss 0.0060 | Win/lose count 9.0/4.0 (5.0)
Epoch 010/020 | Loss 0.0015 | Win/lose count 3.0/2.0 (1.0)
Epoch 011/020 | Loss 0.0019 | Win/lose count 5.0/3.0 (2.0)
Epoch 012/020 | Loss 0.0010 | Win/lose count 7.5/1.0 (6.5)
Epoch 013/020 | Loss 0.0039 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/020 | Loss 0.0476 | Win/lose count 9.0/3.0 (6.0)
Epoch 015/020 | Loss 0.0027 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/020 | Loss 0.0564 | Win/lose count 9.5/

Epoch 000/020 | Loss 0.0010 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/020 | Loss 0.0131 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/020 | Loss 0.0097 | Win/lose count 7.0/3.0 (4.0)
Epoch 003/020 | Loss 0.0087 | Win/lose count 1.5/5.0 (-3.5)
Epoch 004/020 | Loss 0.0048 | Win/lose count 10.0/8.0 (2.0)
Epoch 005/020 | Loss 0.0080 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/020 | Loss 0.0067 | Win/lose count 8.0/3.0 (5.0)
Epoch 007/020 | Loss 0.0029 | Win/lose count 10.5/2.0 (8.5)
Epoch 008/020 | Loss 0.0036 | Win/lose count 4.5/1.0 (3.5)
Epoch 009/020 | Loss 0.0044 | Win/lose count 16.0/7.0 (9.0)
Epoch 010/020 | Loss 0.0014 | Win/lose count 5.0/1.0 (4.0)
Epoch 011/020 | Loss 0.0492 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/020 | Loss 0.0051 | Win/lose count 15.5/4.0 (11.5)
Epoch 013/020 | Loss 0.0033 | Win/lose count 7.0/7.0 (0.0)
Epoch 014/020 | Loss 0.0030 | Win/lose count 6.5/1.0 (5.5)
Epoch 015/020 | Loss 0.0036 | Win/lose count 7.0/1.0 (6.0)
Epoch 016/020 | Loss 0.0484 | Win/lose count 7.0/

In [47]:
fc_df_par = pd.DataFrame(fullconnect_res_params)
fc_df_par

,num_layers,hidden_size,learning_rate,batch_size,memory_size,score
0,1,50,0.05,16,1000,3.70
1,1,50,0.05,16,2000,1.95
2,1,50,0.05,16,3000,2.80
3,1,50,0.05,32,1000,1.45
4,1,50,0.05,32,2000,0.70
5,1,50,0.05,32,3000,3.20
6,1,50,0.05,48,1000,4.50
7,1,50,0.05,48,2000,2.10
8,1,50,0.05,48,3000,1.10
9,1,50,0.10,16,1000,2.80


The best result is obtained with the following parameters: learning rate = 0.25, batch size = 32, memory size = 2000. We will use these parameters in next tasks.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(filters=48, kernel_size=2, activation="relu", padding="valid", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(filters=16, kernel_size=2, activation="relu"))
        model.add(Flatten())
        model.add(Dense(self.n_action))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [49]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0054 | Win/lose count 5.5/6.0 (-0.5)
Epoch 001/020 | Loss 0.0017 | Win/lose count 2.0/0 (2.0)
Epoch 002/020 | Loss 0.0463 | Win/lose count 1.5/1.0 (0.5)
Epoch 003/020 | Loss 0.0328 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/020 | Loss 0.0028 | Win/lose count 2.0/8.0 (-6.0)
Epoch 005/020 | Loss 0.0434 | Win/lose count 0/2.0 (-2.0)
Epoch 006/020 | Loss 0.0045 | Win/lose count 5.0/6.0 (-1.0)
Epoch 007/020 | Loss 0.0081 | Win/lose count 7.0/7.0 (0.0)
Epoch 008/020 | Loss 0.0045 | Win/lose count 4.5/0 (4.5)
Epoch 009/020 | Loss 0.0066 | Win/lose count 9.5/1.0 (8.5)
Epoch 010/020 | Loss 0.0023 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/020 | Loss 0.0057 | Win/lose count 7.5/1.0 (6.5)
Epoch 012/020 | Loss 0.0021 | Win/lose count 6.5/4.0 (2.5)
Epoch 013/020 | Loss 0.0021 | Win/lose count 6.5/5.0 (1.5)
Epoch 014/020 | Loss 0.0028 | Win/lose count 7.0/2.0 (5.0)
Epoch 015/020 | Loss 0.0034 | Win/lose count 14.0/5.0 (9.0)
Epoch 016/020 | Loss 0.0026 | Win/lose count 4.5/3.0 (1.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [53]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.5/0. Average score (6.5)
Win/lose count 14.5/2.0. Average score (9.5)
Win/lose count 4.0/1.0. Average score (7.333333333333333)
Win/lose count 12.0/2.0. Average score (8.0)
Win/lose count 7.0/5.0. Average score (6.8)
Win/lose count 10.0/0. Average score (7.333333333333333)
Win/lose count 7.0/1.0. Average score (7.142857142857143)
Win/lose count 3.5/3.0. Average score (6.3125)
Win/lose count 9.5/1.0. Average score (6.555555555555555)
Win/lose count 9.5/1.0. Average score (6.75)
Final score: 6.75
Test of the FC
Win/lose count 8.5/2.0. Average score (6.5)
Win/lose count 6.0/1.0. Average score (5.75)
Win/lose count 5.5/0. Average score (5.666666666666667)
Win/lose count 2.5/2.0. Average score (4.375)
Win/lose count 8.5/2.0. Average score (4.8)
Win/lose count 5.5/1.0. Average score (4.75)
Win/lose count 11.5/4.0. Average score (5.142857142857143)
Win/lose count 12.5/1.0. Average score (5.9375)
Win/lose count 5.5/3.0. Average score (5.555555555555555)
Win/los

5.55

In [55]:
HTML(display_videos('cnn_test0.mp4'))

In [56]:
HTML(display_videos('fc_test10.mp4'))

In [59]:
epochs_train = 20
epochs_test = 10
temperature_res = []
temperatures = [0.1, 0.5, 0.9]

for temp in tqdmnb(temperatures):
    env = Environment(grid_size=size, max_time=T, temperature=temp)

    agent = DQN_FC(size, lr=0.25, epsilon=0.1, memory_size=2000, batch_size=32, hidden_size=50, num_layers=1)
    train(agent, env, epochs_train, prefix='fc_train')
    score = test(agent, env, epochs_test, prefix='fc_test')
    res = {'mode': "FC", 'temperature': temp, 'score': score}
    
    print('result: {}'.format(res))
    temperature_res.append(res)

    agent = DQN_CNN(size, lr=0.25, epsilon=0.1, memory_size=2000, batch_size=32)
    train(agent, env, epochs_train, prefix='cnn_train')
    score = test(agent, env, epochs_test, prefix='cnn_test')
    res = {'mode': "CNN", 'temperature': temp, 'score': score}

    print('result: {}'.format(res))
    temperature_res.append(res)

Epoch 000/020 | Loss 0.0091 | Win/lose count 1.5/3.0 (-1.5)
Epoch 001/020 | Loss 0.0093 | Win/lose count 1.0/3.0 (-2.0)
Epoch 002/020 | Loss 0.0014 | Win/lose count 1.5/1.0 (0.5)
Epoch 003/020 | Loss 0.0027 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/020 | Loss 0.0031 | Win/lose count 1.0/0 (1.0)
Epoch 005/020 | Loss 0.0087 | Win/lose count 0/1.0 (-1.0)
Epoch 006/020 | Loss 0.0046 | Win/lose count 3.0/0 (3.0)
Epoch 007/020 | Loss 0.0153 | Win/lose count 1.0/1.0 (0.0)
Epoch 008/020 | Loss 0.0628 | Win/lose count 1.5/0 (1.5)
Epoch 009/020 | Loss 0.0630 | Win/lose count 0.5/0 (0.5)
Epoch 010/020 | Loss 0.0018 | Win/lose count 1.5/1.0 (0.5)
Epoch 011/020 | Loss 0.0021 | Win/lose count 1.5/2.0 (-0.5)
Epoch 012/020 | Loss 0.0033 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/020 | Loss 0.0017 | Win/lose count 0.5/3.0 (-2.5)
Epoch 014/020 | Loss 0.0012 | Win/lose count 1.0/2.0 (-1.0)
Epoch 015/020 | Loss 0.0076 | Win/lose count 2.5/3.0 (-0.5)
Epoch 016/020 | Loss 0.0442 | Win/lose count 0.5/1.0 (-0.5

In [60]:
temp_df = pd.DataFrame(temperature_res)
temp_df

,mode,temperature,score
0,FC,0.1,2.05
1,CNN,0.1,2.90
2,FC,0.5,9.50
3,CNN,0.5,14.20
4,FC,0.9,16.85
5,CNN,0.9,33.90


* Comparing the performances of both algorithms we can conclude that CNN handles better with this task. 
* The observed issue: both algoritms tends to stuck into small regions making loops in already observed cells. So we need to pay attntion to the exploration component.
* Temperature impact: with temperature increasing, the scores of FC and CNN methods as well as the difference between them increase.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        agent.set_epsilon(agent.epsilon*0.5)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [73]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.25, epsilon=0.6, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, 10, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/010 | Loss 0.0074 | Win/lose count 4.0/11.599999999999982 (-7.599999999999982)
Epoch 001/010 | Loss 0.0077 | Win/lose count 1.3/9.599999999999982 (-8.299999999999981)
Epoch 002/010 | Loss 0.0141 | Win/lose count 2.0/13.399999999999977 (-11.399999999999977)
Epoch 003/010 | Loss 0.0126 | Win/lose count 2.5/13.599999999999984 (-11.099999999999984)
Epoch 004/010 | Loss 0.0019 | Win/lose count 0.8/9.89999999999998 (-9.09999999999998)
Epoch 005/010 | Loss 0.0064 | Win/lose count 3.0/12.199999999999982 (-9.199999999999982)
Epoch 006/010 | Loss 0.0087 | Win/lose count 0/11.999999999999973 (-11.999999999999973)
Epoch 007/010 | Loss 0.0158 | Win/lose count 4.1/12.499999999999984 (-8.399999999999984)
Epoch 008/010 | Loss 0.0152 | Win/lose count 0/11.099999999999977 (-11.099999999999977)
Epoch 009/010 | Loss 0.0257 | Win/lose count 4.1/11.899999999999977 (-7.799999999999978)


In [75]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 9.0/2.0. Average score (7.0)
Win/lose count 0.5/1.0. Average score (3.25)
Win/lose count 7.0/3.0. Average score (3.5)
Win/lose count 1.0/1.0. Average score (2.625)
Win/lose count 3.5/3.0. Average score (2.2)
Win/lose count 3.0/0. Average score (2.3333333333333335)
Win/lose count 1.5/1.0. Average score (2.0714285714285716)
Win/lose count 4.0/4.0. Average score (1.8125)
Win/lose count 1.0/2.0. Average score (1.5)
Win/lose count 2.0/2.0. Average score (1.35)
Final score: 1.35


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***